## xT - Expected Threat tutorial

Expected threat is how we can quantify a players contributions independent of the outcome
#### Articles to read:
    - https://karun.in/blog/expected-threat.html#visualizing-xt
    - https://soccermatics.medium.com/explaining-expected-threat-cbc775d97935a
    - https://theathletic.com/2751525/2021/08/06/introducing-expected-threat-or-xt-the-new-metric-on-the-block/
    - https://sharmaabhishekk.github.io/projects/xt-derivation-julia (if you are interested in the Julia language)
    - https://www.youtube.com/watch?v=RwVLUH4BFfA

In [1]:
import pandas as pd
import numpy as np

# Open the HDF5 file and list the keys
with pd.HDFStore('spadl.h5', mode='r') as store:
    print(store.keys())

# Load the 'events' dataset from the HDF5 file
df = pd.read_hdf("spadl.h5", key="actions/game_2499719")    

# preprocess the dataframe so we only have the data we want
# We want to get it down to be only successful passes made by Bruno Penandes
# we can use pandas .loc method to achieve this with a bunch of & statements


['/actiontypes', '/bodyparts', '/games', '/player_games', '/players', '/results', '/teams', '/actions/game_2499719', '/actions/game_2499720', '/actions/game_2499721', '/actions/game_2499722', '/actions/game_2499723', '/actions/game_2499724', '/actions/game_2499725', '/actions/game_2499726', '/actions/game_2499727', '/actions/game_2499728', '/actions/game_2499729', '/actions/game_2499730', '/actions/game_2499731', '/actions/game_2499732', '/actions/game_2499733', '/actions/game_2499734', '/actions/game_2499735', '/actions/game_2499736', '/actions/game_2499737', '/actions/game_2499738', '/actions/game_2499739', '/actions/game_2499740', '/actions/game_2499741', '/actions/game_2499742', '/actions/game_2499743', '/actions/game_2499744', '/actions/game_2499745', '/actions/game_2499746', '/actions/game_2499747', '/actions/game_2499748', '/actions/game_2499749', '/actions/game_2499750', '/actions/game_2499751', '/actions/game_2499752', '/actions/game_2499753', '/actions/game_2499754', '/action

In [3]:
# Open the HDF5 file and list the keys
with pd.HDFStore('spadl.h5', mode='r') as store:
    print(store.keys())

# Load the 'events' dataset from the HDF5 file
df = pd.read_hdf("spadl.h5", key="players")    

# preprocess the dataframe so we only have the data we want
# We want to get it down to be only successful passes made by Bruno Penandes
# we can use pandas .loc method to achieve this with a bunch of & statements

['/actiontypes', '/bodyparts', '/games', '/player_games', '/players', '/results', '/teams', '/actions/game_2499719', '/actions/game_2499720', '/actions/game_2499721', '/actions/game_2499722', '/actions/game_2499723', '/actions/game_2499724', '/actions/game_2499725', '/actions/game_2499726', '/actions/game_2499727', '/actions/game_2499728', '/actions/game_2499729', '/actions/game_2499730', '/actions/game_2499731', '/actions/game_2499732', '/actions/game_2499733', '/actions/game_2499734', '/actions/game_2499735', '/actions/game_2499736', '/actions/game_2499737', '/actions/game_2499738', '/actions/game_2499739', '/actions/game_2499740', '/actions/game_2499741', '/actions/game_2499742', '/actions/game_2499743', '/actions/game_2499744', '/actions/game_2499745', '/actions/game_2499746', '/actions/game_2499747', '/actions/game_2499748', '/actions/game_2499749', '/actions/game_2499750', '/actions/game_2499751', '/actions/game_2499752', '/actions/game_2499753', '/actions/game_2499754', '/action

In [8]:
df.head(1500)
df.loc[df['first_name']  == 'Kevin']
df['full_name'] = df['first_name']+ ' ' + df['last_name']
df

,player_id,short_name,first_name,last_name,birth_date,full_name
0,32777,H. Tekin,Harun,Tekin,1989-06-17,Harun Tekin
1,393228,M. Sarr,Malang,Sarr,1999-01-23,Malang Sarr
2,393230,O. Mandanda,Over,Mandanda,1998-10-26,Over Mandanda
3,32793,A. N'Diaye,Alfred John Momar,N'Diaye,1990-03-06,Alfred John Momar N'Diaye
4,393247,I. Konaté,Ibrahima,Konaté,1999-05-25,Ibrahima Konaté
...,...,...,...,...,...,...
3598,120839,A. Maâloul,Ali,Maâloul,1990-01-01,Ali Maâloul
3599,114736,C. Cáceda,Carlos Alberto,Cáceda Oyaguez,1991-09-27,Carlos Alberto Cáceda Oyaguez
3600,114908,M. Araujo,Miguel Gianpierre,Araujo Blanco,1994-10-24,Miguel Gianpierre Araujo Blanco
3601,285583,A. Tagnaouti,Ahmed Reda,Tagnaouti,1996-04-05,Ahmed Reda Tagnaouti


In [2]:
# Load the 'events' dataset from the HDF5 file
action_types_df = pd.read_hdf("spadl.h5", key="actiontypes")    

In [4]:
action_types_df.head(1500)

,type_id,type_name
0,0,pass
1,1,cross
2,2,throw_in
3,3,freekick_crossed
4,4,freekick_short
5,5,corner_crossed
6,6,corner_short
7,7,take_on
8,8,foul
9,9,tackle


In [41]:
#Import xT Grid, turn it into an array, and then get how many rows and columns it has
xT = pd.read_excel("xTGrid.xlsx", header=None)
xT = np.array(xT)
xT_rows, xT_cols = xT.shape

In [42]:
xT # this is how our pitch is broken up

array([[0.00099757, 0.00161179, 0.0020186 , 0.00260809, 0.00334013,
        0.00429513, 0.0056995 , 0.00766727, 0.01087671, 0.01342074,
        0.01682068, 0.02730302],
       [0.00147651, 0.0024174 , 0.00269069, 0.00304666, 0.00394049,
        0.005071  , 0.00666887, 0.0086421 , 0.01346684, 0.01766815,
        0.02115845, 0.02047978],
       [0.00227074, 0.00276085, 0.00299808, 0.00332199, 0.00422842,
        0.00536492, 0.00728664, 0.009349  , 0.01504358, 0.02793171,
        0.04829261, 0.04985027],
       [0.00489296, 0.00305791, 0.00305669, 0.00356722, 0.00430459,
        0.00532244, 0.00669099, 0.01061993, 0.01479398, 0.03606533,
        0.10936287, 0.24507636],
       [0.006431  , 0.00358863, 0.00302961, 0.00343361, 0.00426655,
        0.0057272 , 0.00735009, 0.00986016, 0.01808548, 0.04074994,
        0.08348444, 0.200372  ],
       [0.00214547, 0.00272925, 0.00300561, 0.00343727, 0.00432131,
        0.00530279, 0.00703521, 0.00939731, 0.01480376, 0.02213348,
        0.0460667 ,

In [43]:
print(xT_rows,xT_cols)

8 12


In [49]:
#Bin Data for xT


# Open the HDF5 file
with pd.HDFStore('spadl.h5', mode='a') as store:
    # Get all action keys (e.g., '/actions/game_2499719')
    action_keys = [key for key in store.keys() if key.startswith('/actions/')]
    
    # Iterate through each match
    for key in action_keys:
        # Load the action data for this game
        df_match = store[key]
        
        # Extract the game ID from the key (e.g., '2499719')
        game_id = key.split('/')[-1].replace('game_', '')
        
        # Process the match data
        print(f"Processing Game ID: {game_id}")
        print(f"Number of actions: {len(df_match)}")
        print(df_match.head(), "\n")
        # We use a pandas method called .cut --> https://pandas.pydata.org/docs/reference/api/pandas.cut.html
        # We say labels=False because we want the integer locator of the bin
        df_match['x1_bin'] = pd.cut(df_match['start_x'], bins=xT_cols, labels=False)
        df_match['y1_bin'] = pd.cut(df_match['start_y'], bins=xT_rows, labels=False)
        df_match['x2_bin'] = pd.cut(df_match['end_x'], bins=xT_cols, labels=False)
        df_match['y2_bin'] = pd.cut(df_match['end_y'], bins=xT_rows, labels=False)

        # Save the updated DataFrame back to the same key
        store.put(key, df_match, format='table', data_columns=True)
            
            # Verify the save (optional)
        print(f"Saved updated data for Game ID: {game_id}")
        print(store[key][['x1_bin', 'y1_bin', 'x2_bin', 'y2_bin']].head(), "\n")



Processing Game ID: 2499719
Number of actions: 1288
     game_id  period_id  time_seconds  team_id  player_id  start_x  start_y  \
0  2499719.0        1.0      2.758649   1609.0    25413.0    51.45    34.68   
1  2499719.0        1.0      4.946850   1609.0   370224.0    32.55    14.96   
2  2499719.0        1.0      6.542188   1609.0     3319.0    53.55    17.00   
3  2499719.0        1.0      8.143395   1609.0   120339.0    36.75    19.72   
4  2499719.0        1.0     10.302366   1609.0   167145.0    43.05     3.40   

   end_x  end_y  bodypart_id  type_id  result_id  
0  32.55  14.96            0        0          1  
1  53.55  17.00            0        0          1  
2  36.75  19.72            1        0          1  
3  43.05   3.40            1        0          1  
4  75.60   8.16            0        0          1   

Saved updated data for Game ID: 2499719
   x1_bin  y1_bin  x2_bin  y2_bin
0       5       4       3       1
1       3       1       6       1
2       6       1      

In [58]:
# Load the 'events' dataset from the HDF5 file
df = pd.read_hdf("spadl.h5", key="actions/game_2499719")    

In [60]:
import pandas as pd

# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Auto-detect terminal width
pd.set_option('display.max_colwidth', None)  # Show full column content
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping to multiple lines
pd.set_option('display.width', 1000)  # Increase terminal display width

# Now when you print your DataFrame, it won't be truncated
print(df)

        game_id  period_id  time_seconds  team_id  player_id  start_x  start_y   end_x  end_y  bodypart_id  type_id  result_id  x1_bin  y1_bin  x2_bin  y2_bin  xT_Previous  xT_Current
0     2499719.0        1.0      2.758649   1609.0    25413.0    51.45    34.68   32.55  14.96            0        0          1       5       4       3       1     0.005727    0.003047
1     2499719.0        1.0      4.946850   1609.0   370224.0    32.55    14.96   53.55  17.00            0        0          1       3       1       6       1     0.003047    0.006669
2     2499719.0        1.0      6.542188   1609.0     3319.0    53.55    17.00   36.75  19.72            1        0          1       6       1       4       2     0.006669    0.004228
3     2499719.0        1.0      8.143395   1609.0   120339.0    36.75    19.72   43.05   3.40            1        0          1       4       2       4       0     0.004228    0.003340
4     2499719.0        1.0     10.302366   1609.0   167145.0    43.05     3.40  

In [51]:
df.head()

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,bodypart_id,type_id,result_id,x1_bin,y1_bin,x2_bin,y2_bin
0,2499719.0,1.0,2.758649,1609.0,25413.0,51.45,34.68,32.55,14.96,0,0,1,5,4,3,1
1,2499719.0,1.0,4.946850,1609.0,370224.0,32.55,14.96,53.55,17.00,0,0,1,3,1,6,1
2,2499719.0,1.0,6.542188,1609.0,3319.0,53.55,17.00,36.75,19.72,1,0,1,6,1,4,2
3,2499719.0,1.0,8.143395,1609.0,120339.0,36.75,19.72,43.05,3.40,1,0,1,4,2,4,0
4,2499719.0,1.0,10.302366,1609.0,167145.0,43.05,3.40,75.60,8.16,0,0,1,4,0,8,0


In [53]:
# Return Bin Values for xT
# We use a lambda function to assign the values from the bins to the x and y values with numpy array slicing
# it is going through each row and assigning the values based on where those numbers are in the numpy array
# kind of an advanced topic, so be sure to familiarize yourself with slicing and indexing
# its basically doing this: xT[4][5] where 4 is the fourth list and 5 is the fifth item in that list


# Open the HDF5 file
with pd.HDFStore('spadl.h5', mode='a') as store:
    # Get all action keys (e.g., '/actions/game_2499719')
    action_keys = [key for key in store.keys() if key.startswith('/actions/')]
    
    # Iterate through each match
    for key in action_keys:
        # Load the action data for this game
        df_match = store[key]
        
        # Extract the game ID from the key (e.g., '2499719')
        game_id = key.split('/')[-1].replace('game_', '')
        
        # Process the match data
        print(f"Processing Game ID: {game_id}")
        print(f"Number of actions: {len(df_match)}")
        print(df_match.head(), "\n")
        # We use a pandas method called .cut --> https://pandas.pydata.org/docs/reference/api/pandas.cut.html
        # We say labels=False because we want the integer locator of the bin

        df_match['xT_Previous'] = df_match[['x1_bin', 'y1_bin']].apply(lambda x: xT[x[1]][x[0]], axis=1)
        df_match['xT_Current'] = df_match[['x2_bin', 'y2_bin']].apply(lambda x: xT[x[1]][x[0]], axis=1)

        # Save the updated DataFrame back to the same key
        store.put(key, df_match, format='table', data_columns=True)
            
            # Verify the save (optional)
        print(f"Saved updated data for Game ID: {game_id}")
        print(store[key][['x1_bin', 'y1_bin', 'x2_bin', 'y2_bin']].head(), "\n")

Processing Game ID: 2499719
Number of actions: 1288
     game_id  period_id  time_seconds  team_id  player_id  start_x  start_y  \
0  2499719.0        1.0      2.758649   1609.0    25413.0    51.45    34.68   
1  2499719.0        1.0      4.946850   1609.0   370224.0    32.55    14.96   
2  2499719.0        1.0      6.542188   1609.0     3319.0    53.55    17.00   
3  2499719.0        1.0      8.143395   1609.0   120339.0    36.75    19.72   
4  2499719.0        1.0     10.302366   1609.0   167145.0    43.05     3.40   

   end_x  end_y  bodypart_id  type_id  result_id  x1_bin  y1_bin  x2_bin  \
0  32.55  14.96            0        0          1       5       4       3   
1  53.55  17.00            0        0          1       3       1       6   
2  36.75  19.72            1        0          1       6       1       4   
3  43.05   3.40            1        0          1       4       2       4   
4  75.60   8.16            0        0          1       4       0       8   

   y2_bin  
0   

C:\Users\midos\AppData\Local\Temp\ipykernel_33980\354058352.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_match['xT_Previous'] = df_match[['x1_bin', 'y1_bin']].apply(lambda x: xT[x[1]][x[0]], axis=1)
C:\Users\midos\AppData\Local\Temp\ipykernel_33980\354058352.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_match['xT_Current'] = df_match[['x2_bin', 'y2_bin']].apply(lambda x: xT[x[1]][x[0]], axis=1)


Saved updated data for Game ID: 2499719
   x1_bin  y1_bin  x2_bin  y2_bin
0       5       4       3       1
1       3       1       6       1
2       6       1       4       2
3       4       2       4       0
4       4       0       8       0 

Processing Game ID: 2499720
Number of actions: 1276
     game_id  period_id  time_seconds  team_id  player_id  start_x  start_y  \
0  2499720.0        1.0      3.358676   1625.0     8325.0    53.55    36.04   
1  2499720.0        1.0      5.108462   1625.0   105339.0    67.20    34.68   
2  2499720.0        1.0      7.758945   1625.0    70086.0    75.60    20.40   
3  2499720.0        1.0     11.267789   1625.0     8307.0    75.60    53.72   
4  2499720.0        1.0     15.599964   1625.0    70086.0    75.60    16.32   

   end_x  end_y  bodypart_id  type_id  result_id  x1_bin  y1_bin  x2_bin  \
0   67.2  34.68            0        0          1       6       4       7   
1   75.6  20.40            0        0          1       7       4       8   

C:\Users\midos\AppData\Local\Temp\ipykernel_33980\354058352.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_match['xT_Previous'] = df_match[['x1_bin', 'y1_bin']].apply(lambda x: xT[x[1]][x[0]], axis=1)
C:\Users\midos\AppData\Local\Temp\ipykernel_33980\354058352.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_match['xT_Current'] = df_match[['x2_bin', 'y2_bin']].apply(lambda x: xT[x[1]][x[0]], axis=1)


Saved updated data for Game ID: 2499720
   x1_bin  y1_bin  x2_bin  y2_bin
0       6       4       7       4
1       7       4       8       2
2       8       2       8       6
3       8       6       8       1
4       8       1       7       4 

Processing Game ID: 2499721
Number of actions: 1169
     game_id  period_id  time_seconds  team_id  player_id  start_x  start_y  \
0  2499721.0        1.0      2.608814   1646.0     8980.0    53.55    34.00   
1  2499721.0        1.0      6.245466   1646.0     9433.0    72.45    39.44   
2  2499721.0        1.0      9.077854   1610.0    14748.0    24.15    15.64   
3  2499721.0        1.0      9.077854   1610.0    14748.0    24.15    15.64   
4  2499721.0        1.0     10.211311   1610.0     3350.0    24.15    16.32   

   end_x  end_y  bodypart_id  type_id  result_id  x1_bin  y1_bin  x2_bin  \
0  72.45  39.44            0        0          1       6       3       8   
1  24.15  15.64            0        0          0       8       4       2   

...

Saved updated data for Game ID: 2500094
   x1_bin  y1_bin  x2_bin  y2_bin
0       6       3       6       3
1       6       3       7       5
2       7       5       7       3
3       7       3       7       4
4       7       4       7       5 

Processing Game ID: 2500095
Number of actions: 1239
     game_id  period_id  time_seconds  team_id  player_id  start_x  start_y  \
0  2500095.0        1.0      4.355220   1625.0    38021.0    55.65    27.88   
1  2500095.0        1.0      7.121227   1625.0     9380.0    69.30    34.00   
2  2500095.0        1.0     10.645127   1619.0     8136.0    22.05     6.12   
3  2500095.0        1.0     22.796050   1619.0    70389.0    11.55     0.00   
4  2500095.0        1.0     24.996773   1619.0      383.0    32.55    16.32   

   end_x  end_y  bodypart_id  type_id  result_id  x1_bin  y1_bin  x2_bin  \
0  69.30  34.00            0        0          1       6       3       7   
1  22.05   6.12            0        0          1       7       3       2   

C:\Users\midos\AppData\Local\Temp\ipykernel_33980\354058352.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_match['xT_Previous'] = df_match[['x1_bin', 'y1_bin']].apply(lambda x: xT[x[1]][x[0]], axis=1)
C:\Users\midos\AppData\Local\Temp\ipykernel_33980\354058352.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_match['xT_Current'] = df_match[['x2_bin', 'y2_bin']].apply(lambda x: xT[x[1]][x[0]], axis=1)
C:\Users\midos\AppData\Local\Temp\ipykernel_33980\354058352.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as la

Saved updated data for Game ID: 2500096
   x1_bin  y1_bin  x2_bin  y2_bin
0       6       3       4       4
1       4       4       3       5
2       3       5       4       4
3       4       4       3       5
4       3       5       5       6 

Processing Game ID: 2500097
Number of actions: 1075
     game_id  period_id  time_seconds  team_id  player_id  start_x  start_y  \
0  2500097.0        1.0      1.601308   1624.0    25804.0    52.50    34.00   
1  2500097.0        1.0      4.275583   1624.0       36.0    33.60    27.88   
2  2500097.0        1.0     12.189233   1631.0   149019.0    80.85    64.60   
3  2500097.0        1.0     15.504551   1624.0       36.0    30.45    33.32   
4  2500097.0        1.0     21.198968   1624.0   210044.0    12.60    65.96   

   end_x  end_y  bodypart_id  type_id  result_id  x1_bin  y1_bin  x2_bin  \
0  33.60  27.88            0        0          1       5       3       3   
1  73.50  51.68            0        0          1       3       3       8   

C:\Users\midos\AppData\Local\Temp\ipykernel_33980\354058352.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_match['xT_Current'] = df_match[['x2_bin', 'y2_bin']].apply(lambda x: xT[x[1]][x[0]], axis=1)
C:\Users\midos\AppData\Local\Temp\ipykernel_33980\354058352.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_match['xT_Previous'] = df_match[['x1_bin', 'y1_bin']].apply(lambda x: xT[x[1]][x[0]], axis=1)
C:\Users\midos\AppData\Local\Temp\ipykernel_33980\354058352.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as la

Saved updated data for Game ID: 2500097
   x1_bin  y1_bin  x2_bin  y2_bin
0       5       3       3       3
1       3       3       8       6
2       9       7       3       3
3       3       3       1       7
4       1       7       3       7 

Processing Game ID: 2500098
Number of actions: 1145
     game_id  period_id  time_seconds  team_id  player_id  start_x  start_y  \
0  2500098.0        1.0      3.215711   1633.0     8561.0    51.45    34.00   
1  2500098.0        1.0      6.071036   1633.0     8582.0    27.30    51.68   
2  2500098.0        1.0      9.592420   1633.0    26499.0    55.65    64.60   
3  2500098.0        1.0      9.962099   1623.0   353236.0    55.65    64.60   
4  2500098.0        1.0     12.574680   1633.0    26499.0    64.05    63.92   

   end_x  end_y  bodypart_id  type_id  result_id  x1_bin  y1_bin  x2_bin  \
0  27.30  51.68            0        0          1       5       3       3   
1  55.65  64.60            0        0          1       3       6       6   

In [54]:
# Load the 'events' dataset from the HDF5 file
df = pd.read_hdf("spadl.h5", key="actions/game_2499719")    

In [55]:
df.head()

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,bodypart_id,type_id,result_id,x1_bin,y1_bin,x2_bin,y2_bin,xT_Previous,xT_Current
0,2499719.0,1.0,2.758649,1609.0,25413.0,51.45,34.68,32.55,14.96,0,0,1,5,4,3,1,0.005727,0.003047
1,2499719.0,1.0,4.946850,1609.0,370224.0,32.55,14.96,53.55,17.00,0,0,1,3,1,6,1,0.003047,0.006669
2,2499719.0,1.0,6.542188,1609.0,3319.0,53.55,17.00,36.75,19.72,1,0,1,6,1,4,2,0.006669,0.004228
3,2499719.0,1.0,8.143395,1609.0,120339.0,36.75,19.72,43.05,3.40,1,0,1,4,2,4,0,0.004228,0.003340
4,2499719.0,1.0,10.302366,1609.0,167145.0,43.05,3.40,75.60,8.16,0,0,1,4,0,8,0,0.003340,0.010877


In [61]:

# Define action types for same-team vs. different-team
same_team_actions = {0, 1, 2, 3, 4, 5, 6, 7, 14, 15, 16, 17, 18, 20, 21, 22}  # From the table
different_team_actions = {8, 9, 10, 11, 12, 13, 19}

# Open the HDF5 file
# Open the HDF5 file in append mode (read/write)
with pd.HDFStore('spadl.h5', mode='a') as store:
    # Get all action keys
    action_keys = [key for key in store.keys() if key.startswith('/actions/')]
    
    # Iterate through each match
    for key in action_keys:
        try:
            # Load the action data for this game
            df_match = store[key].copy()
            
            # Extract the game ID
            game_id = key.split('/')[-1].replace('game_', '')
            
            # Verify required columns
            required_cols = ['start_x', 'start_y', 'end_x', 'end_y', 'team_id', 'type_id']
            missing_cols = [col for col in required_cols if col not in df_match.columns]
            if missing_cols:
                print(f"Game ID: {game_id} - Missing columns: {missing_cols}")
                continue
            
            # Process the match data
            print(f"Processing Game ID: {game_id}")
            print(f"Number of actions: {len(df_match)}")
            print(df_match.head(), "\n")
            

            
            # Determine if the action involves same team or different teams
            # First, check based on action type
            df_match['same_team_action'] = df_match['type_id'].isin(same_team_actions)
            
            # Also check team_id changes between consecutive actions
            df_match['same_team'] = df_match['team_id'] == df_match['team_id'].shift(1)
            
            # Combine: same_team_action AND same_team (both must be true for subtraction)
            df_match['use_subtraction'] = df_match['same_team_action'] & df_match['same_team']
            
            # Calculate xT based on the rule
            df_match['xT'] = np.where(
                df_match['use_subtraction'],
                df_match['xT_Current'] - df_match['xT_Previous'],  # Same team: subtract
                df_match['xT_Current'] + df_match['xT_Previous']   # Different teams: add
            )
            
            # Save the updated DataFrame back to the same key
            store.put(key, df_match, format='table', data_columns=True)
            
            # Verify the save
            print(f"Saved updated data for Game ID: {game_id}")
            print(store[key][['x1_bin', 'y1_bin', 'x2_bin', 'y2_bin', 'xT']].head(), "\n")
            
        except Exception as e:
            print(f"Error processing Game ID: {game_id} - {str(e)}")

Processing Game ID: 2499719
Number of actions: 1288
     game_id  period_id  time_seconds  team_id  player_id  start_x  start_y  end_x  end_y  bodypart_id  type_id  result_id  x1_bin  y1_bin  x2_bin  y2_bin  xT_Previous  xT_Current
0  2499719.0        1.0      2.758649   1609.0    25413.0    51.45    34.68  32.55  14.96            0        0          1       5       4       3       1     0.005727    0.003047
1  2499719.0        1.0      4.946850   1609.0   370224.0    32.55    14.96  53.55  17.00            0        0          1       3       1       6       1     0.003047    0.006669
2  2499719.0        1.0      6.542188   1609.0     3319.0    53.55    17.00  36.75  19.72            1        0          1       6       1       4       2     0.006669    0.004228
3  2499719.0        1.0      8.143395   1609.0   120339.0    36.75    19.72  43.05   3.40            1        0          1       4       2       4       0     0.004228    0.003340
4  2499719.0        1.0     10.302366   1609.0  

In [165]:
# now we find the difference between the start and end to get the xT value
df['xT'] = df['end_zone_value'] - df['start_zone_value']

In [62]:
# Load the 'events' dataset from the HDF5 file
df = pd.read_hdf("spadl.h5", key="actions/game_2499719")    

In [63]:
df.head()

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,bodypart_id,type_id,result_id,x1_bin,y1_bin,x2_bin,y2_bin,xT_Previous,xT_Current,same_team_action,same_team,use_subtraction,xT
0,2499719.0,1.0,2.758649,1609.0,25413.0,51.45,34.68,32.55,14.96,0,0,1,5,4,3,1,0.005727,0.003047,True,False,False,0.008774
1,2499719.0,1.0,4.946850,1609.0,370224.0,32.55,14.96,53.55,17.00,0,0,1,3,1,6,1,0.003047,0.006669,True,True,True,0.003622
2,2499719.0,1.0,6.542188,1609.0,3319.0,53.55,17.00,36.75,19.72,1,0,1,6,1,4,2,0.006669,0.004228,True,True,True,-0.002440
3,2499719.0,1.0,8.143395,1609.0,120339.0,36.75,19.72,43.05,3.40,1,0,1,4,2,4,0,0.004228,0.003340,True,True,True,-0.000888
4,2499719.0,1.0,10.302366,1609.0,167145.0,43.05,3.40,75.60,8.16,0,0,1,4,0,8,0,0.003340,0.010877,True,True,True,0.007537


In [14]:
# Open the HDF5 file
# Open the HDF5 file in append mode (read/write)
with pd.HDFStore('spadl.h5', mode='a') as store:
    # Read players data
    players_data = store['players']  # This may already return a DataFrame
    
    # If it's not already a DataFrame (e.g., if it's a Dataset):
    if not isinstance(players_data, pd.DataFrame):
        players_data = pd.DataFrame(players_data[:])
    
    # Create full name column
    players_data['full_name'] = players_data['first_name'] + ' ' + players_data['last_name']
    
    # Save back to HDF5 if needed
    store.put('players', players_data, format='table')
    


In [15]:
# Load the 'events' dataset from the HDF5 file
df = pd.read_hdf("spadl.h5", key="players")    
df.head(1500)

,player_id,short_name,first_name,last_name,birth_date,full_name
0,32777,H. Tekin,Harun,Tekin,1989-06-17,Harun Tekin
1,393228,M. Sarr,Malang,Sarr,1999-01-23,Malang Sarr
2,393230,O. Mandanda,Over,Mandanda,1998-10-26,Over Mandanda
3,32793,A. N'Diaye,Alfred John Momar,N'Diaye,1990-03-06,Alfred John Momar N'Diaye
4,393247,I. Konaté,Ibrahima,Konaté,1999-05-25,Ibrahima Konaté
...,...,...,...,...,...,...
1495,20468,E. Giaccherini,Emanuele,Giaccherini,1985-05-05,Emanuele Giaccherini
1496,20470,C. Marchisio,Claudio,Marchisio,1986-01-19,Claudio Marchisio
1497,20472,S. Padoin,Simone,Padoin,1984-03-18,Simone Padoin
1498,396628,N. Basin,Nicolas,Basin,1998-07-22,Nicolas Basin


In [16]:
# Open the HDF5 file
with pd.HDFStore('spadl.h5', mode='a') as store:
    # Get all action keys (e.g., '/actions/game_2499719')
    action_keys = [key for key in store.keys() if key.startswith('/actions/')]
    
    # Iterate through each match
    for key in action_keys:
        # Load the action data for this game
        df_match = store[key]

        # Read players data
        players_data = store['players']  # This may already return a DataFrame

            # If it's not already a DataFrame (e.g., if it's a Dataset):
        if not isinstance(players_data, pd.DataFrame):
            players_data = pd.DataFrame(players_data[:])
        
        # Extract the game ID from the key (e.g., '2499719')
        game_id = key.split('/')[-1].replace('game_', '')
        
        # Process the match data
        print(f"Processing Game ID: {game_id}")
        print(f"Number of actions: {len(df_match)}")
        print(df_match.head(), "\n")
        # We use a pandas method called .cut --> https://pandas.pydata.org/docs/reference/api/pandas.cut.html

        # Merge keeping only player_id and first_name
        result = df_match.merge(players_data[['player_id', 'full_name']],on='player_id',how='left')

        print(result.head(), "\n")

        # Save the updated DataFrame back to the same key
        store.put(key, result, format='table', data_columns=True)
            
            # Verify the save (optional)
        print(f"Saved updated data for Game ID: {game_id}")
        

Processing Game ID: 2499719
Number of actions: 1288
     game_id  period_id  time_seconds  team_id  player_id  start_x  start_y  \
0  2499719.0        1.0      2.758649   1609.0    25413.0    51.45    34.68   
1  2499719.0        1.0      4.946850   1609.0   370224.0    32.55    14.96   
2  2499719.0        1.0      6.542188   1609.0     3319.0    53.55    17.00   
3  2499719.0        1.0      8.143395   1609.0   120339.0    36.75    19.72   
4  2499719.0        1.0     10.302366   1609.0   167145.0    43.05     3.40   

   end_x  end_y  bodypart_id  ...  x1_bin  y1_bin  x2_bin  y2_bin  \
0  32.55  14.96            0  ...       5       4       3       1   
1  53.55  17.00            0  ...       3       1       6       1   
2  36.75  19.72            1  ...       6       1       4       2   
3  43.05   3.40            1  ...       4       2       4       0   
4  75.60   8.16            0  ...       4       0       8       0   

   xT_Previous  xT_Current  same_team_action  same_team  u

In [17]:
# Load the 'events' dataset from the HDF5 file
df = pd.read_hdf("spadl.h5", key="actions/game_2499719")    

In [18]:
df.head(1500)

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,bodypart_id,...,y1_bin,x2_bin,y2_bin,xT_Previous,xT_Current,same_team_action,same_team,use_subtraction,xT,full_name
0,2499719.0,1.0,2.758649,1609.0,25413.0,51.45,34.68,32.55,14.96,0,...,4,3,1,0.005727,0.003047,True,False,False,0.008774,Alexandre Lacazette
1,2499719.0,1.0,4.946850,1609.0,370224.0,32.55,14.96,53.55,17.00,0,...,1,6,1,0.003047,0.006669,True,True,True,0.003622,Rob Holding
2,2499719.0,1.0,6.542188,1609.0,3319.0,53.55,17.00,36.75,19.72,1,...,1,4,2,0.006669,0.004228,True,True,True,-0.002440,Mesut Özil
3,2499719.0,1.0,8.143395,1609.0,120339.0,36.75,19.72,43.05,3.40,1,...,2,4,0,0.004228,0.003340,True,True,True,-0.000888,Mohamed Naser Elsayed Elneny
4,2499719.0,1.0,10.302366,1609.0,167145.0,43.05,3.40,75.60,8.16,0,...,0,8,0,0.003340,0.010877,True,True,True,0.007537,Héctor Bellerín Moruno
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1283,2499719.0,2.0,2990.768201,1631.0,8653.0,93.45,46.24,97.65,36.04,0,...,5,11,4,0.046067,0.200372,True,False,False,0.246439,Harry Maguire
1284,2499719.0,2.0,2992.491575,1631.0,8480.0,97.65,36.04,56.70,33.32,0,...,4,6,3,0.200372,0.006691,True,True,True,-0.193681,Kasper Schmeichel
1285,2499719.0,2.0,2994.900590,1609.0,49876.0,56.70,33.32,76.65,28.56,1,...,3,8,3,0.006691,0.014794,True,False,False,0.021485,Granit Xhaka
1286,2499719.0,2.0,2997.086392,1609.0,7870.0,76.65,28.56,105.00,27.20,0,...,3,11,3,0.014794,0.245076,False,True,False,0.259870,Aaron Ramsey
